In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# Example from: https://justkode.kr/deep-learning/pytorch-cnn/

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        '''
        in_channels: 입력 채널 수
        out_channels: 출력 채널 수
        kernel_size: 커널 사이즈 / int 혹은 tuple
        stride: stride 사이즈 / int 혹은 tuple / Default: 1 
        padding: padding 사이즈/ int 혹은 tuple/ Default: 0
        padding_mode: padding mode Default: 'zeros'
        dilation: 커널 사이 간격 사이즈 / Fill 0 between kernel
        groups: 입력 층의 그룹 수을 설정하여 입력의 채널 수를 그룹 수에 맞게 분류
                그 다음, 출력의 채널 수를 그룹 수에 맞게 분리.
                입력 그룹과 출력 그룹의 짝을 지은 다음 해당 그룹 안에서만 연산이 이루어지게 한다.
        bias: bias 값을 설정 할 지, 말지를 결정/  Default: True
        '''
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(10 * 12 * 12, 50)
        self.fc2 = nn.Linear(50, 10)    
        
        
    def forward(self, x):
        print("Before: ", x.size())
        x = F.relu(self.conv1(x))
        print("After Conv1: ", x.size())
        x = F.relu(self.conv2(x))
        print("After Conv2: ", x.size())
        x = x.view(-1, 10 * 12 * 12)
        print("After Dimension reduction: ", x.size())
        x = F.relu(self.fc1(x))
        print("After FC1: ", x.size())
        x = self.fc2(x)
        print("After FC2: ", x.size())
        return x

In [3]:
cnn = CNN()
# 10: batch size, 1: channel, 20: height, 20: width
output = cnn(torch.randn(10, 1, 20, 20))   

Before:  torch.Size([10, 1, 20, 20])
After Conv1:  torch.Size([10, 3, 16, 16])
After Conv2:  torch.Size([10, 10, 12, 12])
After Dimension reduction:  torch.Size([10, 1440])
After FC1:  torch.Size([10, 50])
After FC2:  torch.Size([10, 10])


In [4]:
# Pooling Layers
class CNN1(nn.Module):
    def __init__(self):
        super(CNN1, self).__init__()
        self.max_pool1 = nn.MaxPool2d(kernel_size=2)
        self.max_pool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(10 * 5 * 5, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        print("Before: ", x.size()) 
        x = F.relu(self.max_pool1(x))
        print("After MaxPool1: ", x.size())
        x = F.relu(self.max_pool2(x))
        print("After MaxPool2: ", x.size())
        x = x.view(-1, 10 * 5 * 5)
        print("After Dimension reduction: ", x.size())
        x = F.relu(self.fc1(x))
        print("After FC1: ", x.size())
        x = self.fc2(x)
        print("After FC2: ", x.size())
        return x

In [5]:
cnn1 = CNN1()
output = cnn1(torch.randn(10, 1, 20, 20))

Before:  torch.Size([10, 1, 20, 20])
After MaxPool1:  torch.Size([10, 1, 10, 10])
After MaxPool2:  torch.Size([10, 1, 5, 5])
After Dimension reduction:  torch.Size([1, 250])
After FC1:  torch.Size([1, 50])
After FC2:  torch.Size([1, 10])


In [6]:
import torch.optim as optim
from torchvision import datasets, transforms

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Available devices ', torch.cuda.device_count())
print('Current cuda device ', torch.cuda.current_device())
print(torch.cuda.get_device_name(device))

Available devices  1
Current cuda device  0
NVIDIA GeForce GTX 1080 Ti


In [9]:
# Train data
train_data = datasets.MNIST('./data/', train=True, download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))
train_loader = torch.utils.data.DataLoader(train_data, batch_size=50, shuffle=True)

test_data = datasets.MNIST('./data/', train=False, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))
test_loader = torch.utils.data.DataLoader(test_data, batch_size=50, shuffle=True)

In [21]:
class CNN2(nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(4 * 4 * 50, 500)
        self.fc2 = nn.Linear(500, 10)    
        
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size = 2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size = 2, stride=2)  
        
        # Batch_size, 50, 4, 4
        x = x.view(-1, 4 * 4 * 50)  
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [24]:
cnn2 = CNN2()
cnn2.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn2.parameters(), lr=0.01)

In [26]:
cnn2.train()
for epoch in range(10):
    for index, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # Initialize gradient
        output = cnn2(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if index % 100 == 0:
            print("loss of {} epoch, {} index: {}".format(epoch, index, loss.item()))
            

loss of 0 epoch, 0 index: 2.286928176879883
loss of 0 epoch, 100 index: 1.3214448690414429
loss of 0 epoch, 200 index: 0.6104733943939209
loss of 0 epoch, 300 index: 0.14879675209522247
loss of 0 epoch, 400 index: 0.33026039600372314
loss of 0 epoch, 500 index: 0.17899209260940552
loss of 0 epoch, 600 index: 0.38695457577705383
loss of 0 epoch, 700 index: 0.17039556801319122
loss of 0 epoch, 800 index: 0.08569011837244034
loss of 0 epoch, 900 index: 0.28515446186065674
loss of 0 epoch, 1000 index: 0.0805310308933258
loss of 0 epoch, 1100 index: 0.12269935756921768
loss of 1 epoch, 0 index: 0.1695977747440338
loss of 1 epoch, 100 index: 0.10537528246641159
loss of 1 epoch, 200 index: 0.28193211555480957
loss of 1 epoch, 300 index: 0.06703651696443558
loss of 1 epoch, 400 index: 0.12038504332304001
loss of 1 epoch, 500 index: 0.21779827773571014
loss of 1 epoch, 600 index: 0.04637257009744644
loss of 1 epoch, 700 index: 0.17446580529212952
loss of 1 epoch, 800 index: 0.09967949241399765


In [28]:
cnn2.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = cnn2(data)
        test_loss += criterion(output, target).item()
        pred = output.argmax(dim=1, keepdim=True)   
        correct += pred.eq(target.view_as(pred)).sum().item()
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



Test set: Average loss: 6.2689, Accuracy: 9895/10000 (99%)

